## Proms/Constraints testing

In [498]:
import sys; sys.path.insert(0,'/Users/ryan/github/prosodic/')
import sys; sys.path.insert(0,'/Users/ryan/github/cadence/')
from cadence.imports import *

In [499]:
# !rm -rf {PATH_DB}

In [500]:
t=Text('Midnight weary kindness dreary in the flesh')
df=t.data()

Iterating over paragraphs: 100%|██████████| 1/1 [00:00<00:00, 39.52it/s]


In [501]:
def apply_combos(df,group1,group2):
    # combo of indices?
    combo_opts = [
        [x for ii,x in grp.groupby(group2)]
        for i,grp in df.groupby(group1)
    ]

    # poss
    for combo in product(*combo_opts):
        if not len(combo): continue
        odf=pd.concat(combo)
        odf['slot_i']=[i+1 for i in range(len(odf))]
        yield odf

def iter_combos(txtdf,num_proc=1):
    for dfi,dfcombo in enumerate(apply_combos(resetindex(txtdf), 'word_i', 'word_ipa_i')):
        yield dfcombo.assign(combo_i=dfi+1)


In [502]:
dfcombo1=next(iter_combos(df))
dfcombo1

,para_i,sent_i,sentpart_i,line_i,word_i,word_pref,word_str,word_tok,word_ipa_i,word_ipa,syll_i,syll_str,syll_ipa,syll_stress,pos_definite,word_nsyll,pos_degree,prom_lstress,pos_number,prom_strength,prom_tstress,is_heavy,prom_weight,prom_pstress,mtree_ishead,dep_type,is_light,is_trough,is_stressed,pos_xpos,pos_prontype,prom_pstrength,is_peak,word_depth,dep_head,word_isfunc,prom_stress,is_unstressed,pos_upos,slot_i,combo_i
0,1,1,1,1,1,,Midnight,midnight,1,'mɪd.`naɪt,1,mi,'mɪd,P,,2,,1.0,Sing,1.0,0.0,0,NaN,0.0,0.0,compound,0,0,1,NNP,,NaN,1,4,3,0,1.0,0,PROPN,1,1
1,1,1,1,1,1,,Midnight,midnight,1,'mɪd.`naɪt,2,dnight,`naɪt,S,,2,,1.0,Sing,0.0,0.0,0,NaN,0.0,0.0,compound,0,1,1,NNP,,NaN,0,4,3,0,0.5,0,PROPN,2,1
2,1,1,1,1,2,,weary,weary,1,'wɪ.riː,1,wea,'wɪ,P,,2,Pos,1.0,,1.0,0.0,0,NaN,0.0,0.0,amod,0,0,1,JJ,,0.0,1,4,3,0,1.0,0,ADJ,3,1
3,1,1,1,1,2,,weary,weary,1,'wɪ.riː,2,ry,riː,U,,2,Pos,NaN,,0.0,NaN,0,NaN,NaN,0.0,amod,0,1,0,JJ,,NaN,0,4,3,0,0.0,1,ADJ,4,1
4,1,1,1,1,3,,kindness,kindness,1,'kaɪnd.nəs,1,kin,'kaɪnd,P,,2,,1.0,Sing,1.0,0.5,0,NaN,1.0,1.0,root,0,0,1,NN,,1.0,1,4,0,0,1.0,0,NOUN,5,1
5,1,1,1,1,3,,kindness,kindness,1,'kaɪnd.nəs,2,dness,nəs,U,,2,,NaN,Sing,0.0,NaN,0,NaN,NaN,1.0,root,0,1,0,NN,,NaN,0,4,0,0,0.0,1,NOUN,6,1
6,1,1,1,1,4,,dreary,dreary,1,'drɪ.riː,1,drea,'drɪ,P,,2,Pos,1.0,,1.0,0.5,0,NaN,0.0,NaN,amod,0,0,1,JJ,,0.0,1,4,3,0,1.0,0,ADJ,7,1
7,1,1,1,1,4,,dreary,dreary,1,'drɪ.riː,2,ry,riː,U,,2,Pos,NaN,,0.0,NaN,0,NaN,NaN,NaN,amod,0,1,0,JJ,,NaN,0,4,3,0,0.0,1,ADJ,8,1
8,1,1,1,1,5,,in,in,1,ɪn,1,in,ɪn,U,,1,,0.0,,NaN,0.0,0,NaN,0.0,NaN,case,0,0,0,IN,,NaN,0,5,7,1,0.0,1,ADP,9,1
10,1,1,1,1,6,,the,the,1,ðə,1,the,ðə,U,Def,1,,0.0,,NaN,0.0,0,NaN,0.0,0.0,det,0,0,0,DT,Art,0.0,0,6,7,1,0.0,1,DET,10,1


In [503]:
def is_ok_parse(parse,maxS=2,maxW=2):
    return ('s'*(maxS+1)) not in parse and ('w'*(maxW+1)) not in parse

def possible_parses(window_len,maxS=2,maxW=2):
    poss = list(product(*[('w','s') for n in range(window_len)]))
    poss = [''.join(x) for x in poss]
    poss = [x for x in poss if is_ok_parse(x)]
    poss = [x for x in poss if len(x)==window_len]
    return poss

allposs=possible_parses(10)
len(allposs)

178

In [504]:
starters=[x[:4] for x in allposs]
Counter(starters)

Counter({'wwsw': 21,
         'wwss': 13,
         'wsww': 13,
         'wsws': 21,
         'wssw': 21,
         'swws': 21,
         'swsw': 21,
         'swss': 13,
         'ssww': 13,
         'ssws': 21})

In [505]:
def no_w_stress(df, is_s):
    return (1-is_s) * df.prom_stress.values
def no_w_stress_l(df, is_s):
    return (1-is_s) * df.prom_lstress.values
def no_w_stress_p(df, is_s):
    return (1-is_s) * df.prom_pstress.values
def no_w_stress_t(df, is_s):
    return (1-is_s) * df.prom_tstress.values

def no_s_unstress(df, is_s):
    return is_s * (1-df.prom_stress.values)
def no_s_unstress_l(df, is_s):
    return is_s * (1-df.prom_lstress.values)
def no_s_unstress_p(df, is_s):
    return is_s * (1-df.prom_pstress.values)
def no_s_unstress_t(df, is_s):
    return is_s * (1-df.prom_tstress.values)


def no_w_peak(df, is_s):
    return (1-is_s) * df.prom_strength.values
def no_w_peak_p(df, is_s):
    return (1-is_s) * df.prom_pstrength.values
def no_s_trough(df, is_s):
    return is_s * (1-df.prom_strength.values)
def no_s_trough_p(df, is_s):
    return is_s * (1-df.prom_pstrength.values)


def is_disyllab_pos(is_s):
    return np.array([
        float(is_s[i] and ((i and is_s[i-1]) or (i+1<len(is_s) and is_s[i+1])))
        for i in range(len(is_s))
    ])

# def w_resolution(df, is_s):
#     print(is_s)
#     print(is_s_di)
#     print()

def f_resolution(df,is_s):
    o=[np.nan]
    for i in range(1,len(df)):
        # for every 2 consecutive sylls...
        s1=is_s[i-1]
        s2=is_s[i]
        if s1!=s2:
            # not disyllabic
            o.append(np.nan)
        else:
            # is disyllabic
            row1=df.iloc[i-1]
            row2=df.iloc[i]

            if (row1.sent_i, row1.word_i) == (row2.sent_i, row2.word_i):
                # is same word. this constraint applies to disyllabic positions crossing word boundaries
                o.append(np.nan)
            else:
                # is different word
                if not row1.word_isfunc or not row2.word_isfunc:
                    o.append(1.0)
                else:
                    o.append(0.0)
    return o

#     for wi,wdf in df.groupby('word_i'):


#     if len(set(df_mpos.word_i))<2: return np.nan # only applies to word-boundaries    
#     # does this apply to weak positions?
#     if df_mpos.parse_syll.iloc[0]=='w': return np.nan # cannot apply to strong positions

#     firstsyll_islight=df_mpos.iloc[0].is_light
#     firstsyll_isstressed=df_mpos.iloc[0].is_stressed
#     if not (firstsyll_islight and firstsyll_isstressed): return weight
#     return 0

# def f_resolution(df_mpos,weight=1):
#     if len(set(df_mpos.word_i))<2: return np.nan # only applies to word-boundaries
#     if df_mpos.parse_pos.iloc[0]=='ss': return weight # cannot apply to strong positions
#     if sum(df_mpos.is_funcword)!=len(df_mpos): return weight
#     return 0

# # if name=='footmin-w-resolution':
# #     if a.wordtoken is not b.wordtoken: return 0 # only applies within word-boundaries
# #     firstsyll_islight=bool(a.feature('prom.weight',True)) == False
# #     firstsyll_isstressed=bool(a.feature('prom.stress',True)) == True
# #     if not (firstsyll_islight and firstsyll_isstressed):
# #         return self.weight

# # if name=='footmin-f-resolution':
# #     if a.word == b.word: return 0 # only applies to word-boundaries
# #     if meterPos.meterVal=='s': return self.weight # cannot apply to strong positions
# #     a_is_fw = bool(a.word.feature('functionword'))
# #     b_is_fw = bool(b.word.feature('functionword'))
# #     if not (a_is_fw and b_is_fw): return self.weight



In [506]:
constraints=[
    no_w_stress,
    # no_w_stress_l,
    # no_w_stress_p,
    no_w_stress_t,
    no_s_unstress,
    # no_s_unstress_l,
    # no_s_unstress_p,
    # no_s_unstress_t,
    no_w_peak,
    no_w_peak_p,
    # no_s_trough,
    # no_s_trough_p,
    f_resolution,
]

In [507]:
def parse_proms(df, mtr):
    mtr=mtr.replace('|','')
    df=df.iloc[:len(mtr)]
    is_s=np.array([float(x=='s') for x in mtr])
    odf=pd.DataFrame()#.assign(slot_i=df.slot_i, meter=list(mtr))
    for cnstr in constraints:
        odf[cnstr.__name__] = cnstr(df, is_s)
    return odf

In [508]:
parse_proms(dfcombo1,'wswswsws').sum()

no_w_stress      4.0
no_w_stress_t    1.0
no_s_unstress    3.5
no_w_peak        4.0
no_w_peak_p      1.0
f_resolution     0.0
dtype: float64

In [509]:
def parse(dfsylls,**kwargs):
    divide_parse_units(dfsylls,**kwargs)
    return pmap_groups(
        parse_unit,
        dfsylls.groupby('unit_i'),
        num_proc=1
    )

def parse_unit(dfunit,maxS=2,maxW=2):
    o=[]
    for dfcombo in iter_combos(dfunit):
        for mtr in possible_parses(len(dfcombo), maxS=maxS, maxW=maxW):
            o+=[parse_proms(dfcombo, mtr)]
    return pd.concat(o) if len(o) else pd.DataFrame()

In [510]:
pos_types=[['w'],['s'],['ww'],['ss']]
# pos_types=['w','s']#,'ww','ss']

def getlenparse(l): return sum(len(x) for x in l)


# def iter_mpos(nsyll, starter=[]):
#     news=[]
#     for pos_type in pos_types:
#         if starter and starter[-1][-1]==pos_type[0][0]: continue
#         new = starter + pos_type
#         # if starter: print(starter[-1][-1], pos_type[0][0], new)
#         #if not is_ok_parse(new): continue
#         if getlenparse(new)<=nsyll:
#             news.append(new)
#     if news: yield news
#     # print('\n'.join('|'.join(x) for x in news))
#     for new in news:
#         yield from iter_mpos(nsyll, starter=new)


def battle_parses(mtrs,min_pos_len=2):
    bounded={}
    for i1,mtr1 in enumerate(mtrs):
        if len(mtr1)<min_pos_len: continue
        mtr1len=getlenparse(mtr1)
        for i2,mtr2 in enumerate(mtrs):
            if i2<=i1: continue
            if len(mtr2)<min_pos_len: continue
            mtr2len=getlenparse(mtr2)
            if mtr1len == mtr2len:
                print(mtr1)
                print(mtr2)

    return mtrs

def iter_mpos(nsyll, starter=[]):
    news=[]
    for pos_type in pos_types:
        if starter and starter[-1][-1]==pos_type[0][0]: continue
        new = starter + pos_type
        # if starter: print(starter[-1][-1], pos_type[0][0], new)
        #if not is_ok_parse(new): continue
        if getlenparse(new)<=nsyll:
            news.append(new)
    
    # news = battle_parses(news)
    if news: yield from news
    # print('\n'.join('|'.join(x) for x in news))
    for new in news: yield from iter_mpos(nsyll, starter=new)


In [511]:
# for npos,nposdf in sorted(dfpars.groupby('npos')):
#     if npos<2: continue
#     display(nposdf)
#     break

In [512]:
scored={}

for nsyll,nsylldf in sorted(dfpars.groupby('nsyll')):
    if nsyll<4: continue
    for meter in nsylldf.meter:
        scored[meter] = parse_proms(dfcombo1, meter).sum()
    break
        

In [513]:
scored.keys()

dict_keys(['w|s|ww|', 'w|s|w|s|', 'w|ss|w|', 's|w|ss|', 's|w|s|w|', 's|ww|s|', 'ww|ss|', 'ww|s|w|', 'ss|ww|', 'ss|w|s|'])

In [514]:
s1=scored['w|s|w|s|']
s2=scored['s|w|s|w|']
s1

no_w_stress      2.0
no_w_stress_t    0.0
no_s_unstress    1.5
no_w_peak        2.0
no_w_peak_p      0.0
f_resolution     0.0
dtype: float64

In [515]:
o=pd.DataFrame({'wsws':s1, 'swsw':s2, 's1-s2':s1-s2, 's2-s1':s2-s1,'s1<s2':s1<s2,'s2<s1':s2<s1})
o

,wsws,swsw,s1-s2,s2-s1,s1<s2,s2<s1
no_w_stress,2.0,0.5,1.5,-1.5,False,True
no_w_stress_t,0.0,0.0,0.0,0.0,False,False
no_s_unstress,1.5,0.0,1.5,-1.5,False,True
no_w_peak,2.0,0.0,2.0,-2.0,False,True
no_w_peak_p,0.0,0.0,0.0,0.0,False,False
f_resolution,0.0,0.0,0.0,0.0,False,False


In [516]:
bounded={}

for k1 in scored:
    for k2 in scored:
        if k1>=k2: continue
        s1=scored[k1]
        s2=scored[k2]
        s1_ever_better_than_s2 = any(s1<s2)
        s2_ever_better_than_s1 = any(s2<s1)

        if s1_ever_better_than_s2 and not s2_ever_better_than_s1:
            # s1 bounds s2
            print(k1,'--bounds-->',k2)
            bounded[k2]=k1
        elif s2_ever_better_than_s1 and not s1_ever_better_than_s2:
            # s2 bounds s1
            print(k2,'--bounds-->',k1)
            bounded[k1]=k2

w|s|ww| --bounds--> w|s|w|s|
s|w|ss| --bounds--> w|s|w|s|
s|w|s|w| --bounds--> s|w|ss|
s|w|ss| --bounds--> ww|ss|
s|w|s|w| --bounds--> w|s|ww|
s|w|s|w| --bounds--> w|s|w|s|
s|w|s|w| --bounds--> w|ss|w|
s|w|s|w| --bounds--> ww|ss|
s|w|s|w| --bounds--> ww|s|w|
w|ss|w| --bounds--> s|ww|s|
s|w|ss| --bounds--> s|ww|s|
s|w|s|w| --bounds--> s|ww|s|
ww|ss| --bounds--> s|ww|s|
ww|s|w| --bounds--> s|ww|s|
ww|ss| --bounds--> w|s|w|s|
ww|s|w| --bounds--> ww|ss|
ww|s|w| --bounds--> w|s|ww|
ww|s|w| --bounds--> w|s|w|s|
ss|ww| --bounds--> w|s|ww|
ss|ww| --bounds--> w|s|w|s|
ss|ww| --bounds--> w|ss|w|
s|w|s|w| --bounds--> ss|ww|
ss|ww| --bounds--> s|ww|s|
ss|ww| --bounds--> ww|ss|
ss|ww| --bounds--> ss|w|s|
ss|w|s| --bounds--> w|s|w|s|
s|w|ss| --bounds--> ss|w|s|
s|w|s|w| --bounds--> ss|w|s|


In [517]:
def get_poss_parses_cleverly(n):
    dfpars = pd.DataFrame([
        dict(
            npos=len(x),
            nsyll=getlenparse(x),
            meter='|'.join(x) + '|',
            #mtr_pos=x,
            #mtr_syll=''.join(x)
        )
        for x in iter_mpos(n)
        if getlenparse(x)<=n
    ])
    return dfpars

def parse_cleverly(dfcombo,min_nsyll=4):
    bounded={}
    dfpars=get_poss_parses_cleverly(len(dfcombo))
    for nsyll,nsylldf in sorted(dfpars.groupby('nsyll')):
        if nsyll<min_nsyll: continue
        print(nsyll, len(bounded))
    
        scored={}
        for meter in nsylldf.meter:
            exclude=False
            for bmeter in bounded:
                if meter.startswith(bmeter):
                    print(f'Excluding parses starting with {bmeter}')
                    exclude=True
                    break
            
            if exclude: continue
            
            scored[meter] = parse_proms(dfcombo, meter)
        
        for mtr1 in scored:
            for mtr2 in scored:
                if mtr1>=mtr2: continue
                s1=scored[mtr1].sum()
                s2=scored[mtr2].sum()
                s1_ever_better_than_s2 = any(s1<s2)
                s2_ever_better_than_s1 = any(s2<s1)
                if s1_ever_better_than_s2 and not s2_ever_better_than_s1:
                    # s1 bounds s2
                    bounded[k2]=k1
                elif s2_ever_better_than_s1 and not s1_ever_better_than_s2:
                    # s2 bounds s1
                    bounded[k1]=k2
    
    return scored

In [346]:
# class ParseTree(object):
#     def __init__(self,df=None):
#         self.df=df
    
#     def extend()

In [232]:
# %%timeit
o=iter_mpos(10)
x=list(o)

In [345]:
possible_metrical_feet(10).reset_index()

,mpos_0,mpos_1,mpos_2,mpos_3,mpos_4,mpos_5,mpos_6,mpos_7,mpos_8,mpos_9,parse,num_pos,num_syll
0,w,s,w,s,w,s,w,s,w,s,wswswswsws,10,10
1,w,s,w,s,w,s,w,s,ww,,wswswswsww,9,10
2,w,s,w,s,w,s,w,ss,w,,wswswswssw,9,10
3,w,s,w,s,w,s,ww,s,w,,wswswswwsw,9,10
4,w,s,w,s,w,s,ww,ss,,,wswswswwss,8,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,ss,ww,ss,w,s,w,s,,,,sswwsswsws,7,10
174,ss,ww,ss,w,s,ww,,,,,sswwsswsww,6,10
175,ss,ww,ss,w,ss,w,,,,,sswwsswssw,6,10
176,ss,ww,ss,ww,s,w,,,,,sswwsswwsw,6,10


In [93]:
oiterr=iter_branching_mpos(dfcombo1)
o=next(oiterr)
o[2]

,pos_degree,pos_case,pos_number,word_tok,sent_i,sentpart_i,line_i,pos_prontype,word_depth,pos_person,word_i,word_str,pos_upos,pos_definite,pos_tense,word_pref,dep_type,pos_xpos,dep_head,pos_mood,pos_verbform,prom_pstress,prom_pstrength,prom_lstress,mtree_ishead,prom_tstress,word_ipa_i,syll_i,word_ipa,word_nsyll,syll_ipa,syll_str,syll_stress,prom_stress,word_isfunc,prom_weight,prom_strength,is_stressed,is_unstressed,is_heavy,is_light,is_peak,is_trough,unit_i,slot_i,combo_i,meter
0,,,,how,1,1,1,Int,5,,1,How,ADV,,,,advmod,WRB,2,,,0.0,0.0,0.0,NaN,0.00,1,1,haʊ,1,haʊ,how,U,0.0,1,NaN,NaN,0,1,0,0,0,0,1,1,1,w
1,Pos,,,heavy,1,1,1,,6,,2,heavy,ADJ,,,,advmod,JJ,5,,,1.0,1.0,1.0,NaN,0.75,1,1,'hɛ.viː,2,'hɛ,hea,P,1.0,0,NaN,1.0,1,0,0,0,1,0,1,2,1,w


In [42]:
parse(df)

Mapping parse_unit [x1]: 100%|██████████| 1/1 [00:01<00:00,  1.24s/it]


,unit_i,slot_i,meter,no_w_stress,no_w_stress_l,no_w_stress_p,no_w_stress_t,no_s_unstress,no_s_unstress_l,no_s_unstress_p,no_s_unstress_t,no_w_peak,no_w_peak_p,no_s_trough,no_s_trough_p,f_resolution
0,1,1,w,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN
1,1,2,w,1.0,1.0,1.0,0.75,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
2,1,3,s,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN
3,1,4,w,1.0,0.0,0.0,0.50,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN
5,1,5,w,1.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,1,6,w,1.0,1.0,0.0,0.75,0.0,0.0,0.0,0.0,1.0,NaN,0.0,NaN,NaN
8,1,7,s,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN
9,1,8,s,0.0,0.0,0.0,0.00,1.0,1.0,1.0,0.5,NaN,NaN,NaN,NaN,1.0
10,1,9,w,0.0,0.0,0.0,0.50,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN


In [32]:
# %%timeit
odf=pd.DataFrame([
    dict(meter=x, viols=y.mean().mean())
    for x,y in parse(df)
]).sort_values('viols')
odf

Mapping parse_unit [x4]:   0%|          | 0/1 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 2)

In [69]:
x

'sswsswssws'

In [67]:
count_empty_vals??

Object `count_empty_vals` not found.


In [ ]:





def no_window(s,badwindow=(1,1)):
    wlen=len(badwindow)
    l=[]
    window=[]
    for x in s:
        window.append(x)
        l.append(int(tuple(window)==badwindow))
        if len(window)>=wlen:window.pop(0)
    return l

def no_clash(dfparse):
    return no_window(dfparse.is_stressed,badwindow=(1,1))
def no_lapse(dfparse):
    return no_window(dfparse.is_stressed,badwindow=(0,0,0))
def no_nonalt(dfparse):
    ww=no_window(dfparse.is_w,badwindow=(1,1))
    ss=no_window(dfparse.is_s,badwindow=(1,1))
    return [1 if x or y else 0 for x,y in zip(ww,ss)]
def no_trochsub(dfparse):
    x=int(dfparse.is_s.iloc[0]==1)
    return [x]+[0 for y in range(len(dfparse)-1)]

def apply_posthoc_constraints(dfparse):
    dfparse['*clash']=[
        x
        for pi,pdf in dfparse.groupby('parse_i')
        for x in no_clash(pdf)
    ]
    dfparse['*lapse']=[
        x
        for pi,pdf in dfparse.groupby('parse_i')
        for x in no_lapse(pdf)
    ]
    dfparse['*nonalt']=[
        x
        for pi,pdf in dfparse.groupby('parse_i')
        for x in no_nonalt(pdf)
    ]


    dfparse[TOTALCOL]=dfparse[[col for col in dfparse.columns if col.startswith('*') and col!=TOTALCOL]].sum(axis=1)



In [ ]:
## Constraints